## Lesson 3: Email Assistant with Semantic Memory

We previously built an email assistant that:
- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings

Now, we'll add memory. 

We'll give the assistant the ability to `remember details from previous emails`. 

<!-- <p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px">  -->
🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Load API tokens for our 3rd party APIs

In [1]:
import os
from dotenv import load_dotenv
print(load_dotenv())

True


------------------------
## Repeat setup from previous lesson

In [2]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [3]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently."
}

In [4]:
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

## Setup Routing Node

In [5]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated
from langchain.chat_models import init_chat_model

In [6]:
llm = init_chat_model("openai:gpt-4o-mini")

In [7]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [8]:
llm_router = llm.with_structured_output(Router)

In [9]:
from prompts import triage_system_prompt, triage_user_prompt

In [10]:
from langchain_core.tools import tool

In [11]:
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"


In [12]:
@tool
def schedule_meeting(
    attendees: list[str], 
    subject: str, 
    duration_minutes: int, 
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"


In [13]:
@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

-----------------------
## 🆕 Define tools for managing memory (semantic memory)

In [14]:
from langgraph.store.memory import InMemoryStore

In [15]:
store = InMemoryStore(
    index={"embed": "openai:text-embedding-3-small"}
)

c:\Users\farza\anaconda3\envs\agentic-mem-env\Lib\site-packages\langgraph\store\base\embed.py:95: LangChainBetaWarning: The function `init_embeddings` is in beta. It is actively being worked on, so the API may change.
  return init_embeddings(embed)


In [16]:
from langmem import create_manage_memory_tool, create_search_memory_tool

`langgraph_user_id`: This will be passed to the tool as part of the runtime configuration. That will be used to namespace the memory so if we are dealing with emails for multiple users, we can easily handle it and have different collections for different users.

In [17]:
manage_memory_tool = create_manage_memory_tool(
    namespace=(
        "email_assistant", 
        "{langgraph_user_id}",
        "collection"
    )
)
search_memory_tool = create_search_memory_tool(
    namespace=(
        "email_assistant",
        "{langgraph_user_id}",
        "collection"
    )
)

In [18]:
print(manage_memory_tool.name)

manage_memory


In [19]:
print(manage_memory_tool.description)

Create, update, or delete persistent MEMORIES to persist across conversations.
Include the MEMORY ID when updating or deleting a MEMORY. Omit when creating a new MEMORY - it will be created for you.
Proactively call this tool when you:

1. Identify a new USER preference.
2. Receive an explicit USER request to remember something or otherwise alter your behavior.
3. Are working and want to record important context.
4. Identify that an existing MEMORY is incorrect or outdated.


args:
- content
- action: update, create, delete, etc.
- id: id of the memory

In [20]:
manage_memory_tool.args

{'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
  'default': None,
  'title': 'Content'},
 'action': {'default': 'create',
  'enum': ['create', 'update', 'delete'],
  'title': 'Action',
  'type': 'string'},
 'id': {'anyOf': [{'format': 'uuid', 'type': 'string'}, {'type': 'null'}],
  'default': None,
  'title': 'Id'}}

In [24]:
search_memory_tool.name

'search_memory'

In [25]:
search_memory_tool.description

'Search your long-term memories for information relevant to your current context.'

args:
- query
- limit: for the number of memories that we want to return
- offset
- filter

In [26]:
search_memory_tool.args

{'query': {'title': 'Query', 'type': 'string'},
 'limit': {'default': 10, 'title': 'Limit', 'type': 'integer'},
 'offset': {'default': 0, 'title': 'Offset', 'type': 'integer'},
 'filter': {'anyOf': [{'type': 'object'}, {'type': 'null'}],
  'default': None,
  'title': 'Filter'}}

Let's modify the agent's prompt and att 4, and 5 to what we saw in the previous section.

In [27]:
agent_system_prompt_memory = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
4. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
5. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [28]:
def create_prompt(state):
    return [
        {
            "role": "system", 
            "content": agent_system_prompt_memory.format(
                instructions=prompt_instructions["agent_instructions"], 
                **profile
            )
        }
    ] + state['messages']

In [29]:
from langgraph.prebuilt import create_react_agent

### 🔄 Updated
Create the list of tools and the react agent (here called response_agent).

In [31]:
tools= [
    write_email, 
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool
]
response_agent = create_react_agent(
    "openai:gpt-4o",
    tools=tools,
    prompt=create_prompt,
    # Use this to ensure the store is passed to the agent 
    store=store
)

In [32]:
config = {"configurable": {"langgraph_user_id": "lance"}}

Now, let's test our 

In [33]:
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "Jim is my friend"}]},
    config=config
)

Below, we can see that the agent calls manage_memory tool with the argument content.

In [34]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Jim is my friend
================================== Ai Message ==================================
Tool Calls:
  manage_memory (call_PFcLaWkFl1qZlKfroTFmGxsq)
 Call ID: call_PFcLaWkFl1qZlKfroTFmGxsq
  Args:
    content: John considers Jim a friend.
    action: create
================================= Tool Message =================================
Name: manage_memory

created memory 8f77f2d7-b0d0-4b77-9a97-74bf9ec2e1b3
================================== Ai Message ==================================

Great! I've noted that Jim is your friend. If there's anything specific you'd like to remember or do related to Jim, just let me know!


In [35]:
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "who is jim?"}]},
    config=config
)

In [36]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

who is jim?
================================== Ai Message ==================================
Tool Calls:
  search_memory (call_9EQkR91xiWE8bUE5rbkLwjAc)
 Call ID: call_9EQkR91xiWE8bUE5rbkLwjAc
  Args:
    query: Jim
================================= Tool Message =================================
Name: search_memory

[{"namespace": ["email_assistant", "lance", "collection"], "key": "8f77f2d7-b0d0-4b77-9a97-74bf9ec2e1b3", "value": {"content": "John considers Jim a friend."}, "created_at": "2025-03-24T03:24:57.588894+00:00", "updated_at": "2025-03-24T03:24:57.588894+00:00", "score": 0.4050819651381967}]
================================== Ai Message ==================================

Jim is considered a friend by John. If you need more specific information, please let me know!


In [37]:
store.list_namespaces()

[('email_assistant', 'lance', 'collection')]

In [38]:
store.search(('email_assistant', 'lance', 'collection'))

[Item(namespace=['email_assistant', 'lance', 'collection'], key='8f77f2d7-b0d0-4b77-9a97-74bf9ec2e1b3', value={'content': 'John considers Jim a friend.'}, created_at='2025-03-24T03:24:57.588894+00:00', updated_at='2025-03-24T03:24:57.588894+00:00', score=None)]

In [39]:
store.search(('email_assistant', 'lance', 'collection'), query="jim")

[Item(namespace=['email_assistant', 'lance', 'collection'], key='8f77f2d7-b0d0-4b77-9a97-74bf9ec2e1b3', value={'content': 'John considers Jim a friend.'}, created_at='2025-03-24T03:24:57.588894+00:00', updated_at='2025-03-24T03:24:57.588894+00:00', score=0.4980010787569783)]

## Create the rest of the agent

Everything up to creating the email agent is the same as before.

In [40]:
from langgraph.graph import add_messages

class State(TypedDict):
    email_input: dict
    messages: Annotated[list, add_messages]

In [41]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing import Literal
from IPython.display import Image, display

In [42]:
def triage_router(state: State) -> Command[
    Literal["response_agent", "__end__"]
]:
    author = state['email_input']['author']
    to = state['email_input']['to']
    subject = state['email_input']['subject']
    email_thread = state['email_input']['email_thread']

    system_prompt = triage_system_prompt.format(
        full_name=profile["full_name"],
        name=profile["name"],
        user_profile_background=profile["user_profile_background"],
        triage_no=prompt_instructions["triage_rules"]["ignore"],
        triage_notify=prompt_instructions["triage_rules"]["notify"],
        triage_email=prompt_instructions["triage_rules"]["respond"],
        examples=None
    )
    user_prompt = triage_user_prompt.format(
        author=author, 
        to=to, 
        subject=subject, 
        email_thread=email_thread
    )
    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )
    if result.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        goto = "response_agent"
        update = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Respond to the email {state['email_input']}",
                }
            ]
        }
    elif result.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
        update = None
        goto = END
    elif result.classification == "notify":
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
        update = None
        goto = END
    else:
        raise ValueError(f"Invalid classification: {result.classification}")
    return Command(goto=goto, update=update)

## Create email agent

- We use the response_agent here
- We create the compile by passing the store=store which gives the system access to the longterm memory store that we created.

In [43]:
email_agent = StateGraph(State)
email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")
email_agent = email_agent.compile(store=store)

The graph is the same as before.

In [ ]:
from IPython.display import Image, display

display(Image(email_agent.get_graph(xray=True).draw_mermaid_png()))

Here we create the email example

In [48]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [49]:
response = email_agent.invoke(
    {"email_input": email_input},
    config=config
)

📧 Classification: RESPOND - This email requires a response


In [50]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi John,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_vZyhQMG2zotxyexl1OBjRqYI)
 Call ID: call_vZyhQMG2zotxyexl1OBjRqYI
  Args:
    to: alice.smith@company.com
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thank you for reaching out about the API documentation. It sounds like there might be an oversight with those endpoints

## Try a follow-up email

In [51]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Follow up",
    "email_thread": """Hi John,

Any update on my previous ask?""",
}

In [52]:
response = email_agent.invoke({"email_input": email_input}, config=config)

📧 Classification: RESPOND - This email requires a response


Remember, query might change in the `search_memory` tool for different tests since this is something that the model will decide.

In [53]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Follow up', 'email_thread': 'Hi John,\n\nAny update on my previous ask?'}
================================== Ai Message ==================================
Tool Calls:
  search_memory (call_ojIPlE5MThgWPhLIlLTJIsh2)
 Call ID: call_ojIPlE5MThgWPhLIlLTJIsh2
  Args:
    query: Alice Smith
    limit: 1
================================= Tool Message =================================
Name: search_memory

[{"namespace": ["email_assistant", "lance", "collection"], "key": "8f77f2d7-b0d0-4b77-9a97-74bf9ec2e1b3", "value": {"content": "John considers Jim a friend."}, "created_at": "2025-03-24T03:24:57.588894+00:00", "updated_at": "2025-03-24T03:24:57.588894+00:00", "score": 0.23295318335053153}]
================================== Ai Message ==================================
Tool Calls:
  write